In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secrt 탭에 `OPENAI_API_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('20250624_skn14_llm')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고, 많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕~ 내 이름은 차은우야~"
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕하세요, 차은우님! 만나서 반가워요. 어떻게 도와드릴까요?'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력 데이터/예시
4. Output Indicator 출력 지시

## 기사 제목 교정

- 기자들이 송고한 기사에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 개꿀 라디오 방송에 주목해주세요.'
title_before = '개빡센 노동으로 끼니 거르기 일쑤인 노동자'

# 역할/페르소나/지시사항
system_message = """
기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사 제목을 읽고 주요내용을 이해하세요.
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

- 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
- 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

### Extra Instructions ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response.choices[0].message.content

'- 원래 제목: "개빡센 노동으로 끼니 거르기 일쑤인 노동자"\n- 교정 제목: "과중한 노동으로 끼니를 챙기지 못하는 노동자"'

In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p = 1):

    # 역할/페르소나/지시사항
    system_message = """
    기자들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사 제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차 회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 간결히 수정해주세요.

    {title_before}
    """

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상, 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: "주말 미친 폭우 예상, 모두들 무사하시길~"
- 교정 제목: "주말 폭우 예상, 모두 안전하길 바랍니다"


## 연애 코치 ReAct

In [5]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p = 1):

    # 역할/페르소나/지시사항
    system_message = """
   <<System Instruction>>
   어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해주세요.

   <<Output Format>>
   1. 상황분석:
   2. 행동계획:
   3. 실행:

    """

    user_message = f"현재상황: {prompt}"

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response.choices[0].message.content

prompt = '제가 어제 친구들이랑 놀다가 여자친구랑 연락을 잘 못해서, 단단히 삐져있는 상태입니다. 이 위기를 어떻게 극복해야 할까요?'
output = dating_coach(prompt)
print(output)

1. 상황분석:
   - 여자친구가 삐져있는 이유는 연락이 원활하지 않아 소외감이나 불안감을 느꼈을 가능성이 큽니다. 친구들과의 시간도 중요하지만, 연애에서는 서로의 소통이 매우 중요하죠. 따라서, 현재 상황에서 여자친구의 감정을 이해하고 그에 대한 적절한 대응이 필요합니다.

2. 행동계획:
   - 진심어린 사과: 여자친구에게 연락이 소홀했던 것에 대해 진심으로 사과합니다. 그와 동시에 그녀의 기분을 이해하고 있다는 것을 표현해야 합니다.
   - 상대의 감정에 공감: "어제 연락이 안 돼서 많이 걱정했겠다"라며 그녀의 기분을 이해하고 걱정했던 점에 공감하는 말을 전하며 그녀의 감정을 존중합니다.
   - 소통의 시간을 가지기: 사과 후, 직접 만나거나 통화로 진솔한 대화를 나누는 시간을 가지며, 그녀가 느꼈던 감정에 대해 들어봅니다.

3. 실행:
   - 즉시 연락을 시도해 보세요. 문자로 간단한 사과와 함께 대화하고 싶은 의사를 나타내고, "우리 얘기 조금 할 시간을 가질 수 있을까?"라고 제안합니다.
   - 약속이 잡히면, 최대한 진지하고 진솔한 태도로 그녀의 이야기를 들어보며, 그녀가 어떻게 느꼈는지 존중해 주세요. 자신의 입장도 솔직하게 이야기하며 서로의 이해를 돕는 것이 중요합니다.
   - 이후에는 서로 일정 공유하며 소통을 강화하는 방법을 논의해 보세요. 예를 들어, 일정이 있을 때에도 간단한 메세지를 통해 서로의 상태를 확인하는 습관을 만들어보는 것도 좋습니다. 

이 과정을 통해 서로의 감정을 이해하며 더욱 가까워질 수 있을 것입니다.


## 냉털마스터 ReAct

- 사용자는 냉장고에 남아있는 음식 재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해줌

- Reasoning/Action을 끌어낼 수 있는 적절한 프롬프팅을 작성하세요

In [11]:
def food_coach(prompt, model='gpt-4o-mini', temperature=1, top_p = 1):

    # 역할/페르소나/지시사항
    system_message = """
   <<System Instruction>>
   주어진 음식 재료로 만들 수 있는 음식 2~3가지를 소개해주세요

   <<Output Format>>
   1.음식명
   - 재료:
   - 만드는 방법:
    만드는 방법은 1) 2) 3) 이런식으로 표현해주세요
    """

    user_message = f"냉장고 속 재료: {prompt}"

    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": system_message
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": user_message
            }
        ]
        }
    ],
    response_format={
        "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
    )

    return response.choices[0].message.content

prompt = '파, 양파, 당근, 베이컨, 삼겹살, 김치'
output = food_coach(prompt)
print(output)

1. 김치 볶음밥
   - 재료: 밥, 김치, 양파, 당근, 베이컨, 파
   - 만드는 방법:
     1) 팬에 베이컨을 넣고 적당히 볶아 기름을 내줍니다.
     2) 양파와 당근을 잘게 썰어 팬에 넣고 볶습니다.
     3) 김치를 넣고 함께 볶은 후, 밥을 추가해 고루 섞이도록 볶읍니다.
     4) 마지막으로 다져놓은 파를 넣고 한 번 더 볶아 완성합니다.

2. 삼겹살 김치찜
   - 재료: 삼겹살, 김치, 양파, 파, 당근
   - 만드는 방법:
     1) 냄비에 삼겹살을 넣고 겉면을 노릇노릇하게 구워냅니다.
     2) 잘라놓은 양파, 당근, 김치를 넣고 재료가 잘 섞이도록 볶습니다.
     3) 물을 적당히 부어 끓이면서 뚜껑을 덮고 중불로 30분 정도 익힙니다.
     4) 마지막으로 썰어놓은 파를 위에 올리고 잠깐 더 끓이면 완성입니다.

3. 베이컨 파전
   - 재료: 파, 양파, 당근, 베이컨
   - 만드는 방법:
     1) 파, 양파, 당근을 채썰어 볼에 담습니다.
     2) 베이컨도 잘게 썰어 함께 넣고 반죽을 위해 밀가루와 물을 추가합니다.
     3) 팬에 기름을 두르고 반죽을 얇게 펴서 양면을 노릇하게 부칩니다.
     4) 완성된 파전을 먹기 좋은 크기로 잘라서 제공합니다.


## 면접 질문 생성 JSON 출력

In [14]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content
jon_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |

"""
output = job_interview(jon_posting)
print(output)

{
    "hard_skill": [
        {
            "question": "Python 프로그래밍 언어를 사용하여 머신러닝 모델을 배포하기 위한 코드를 어떻게 작성하겠습니까?",
            "answer": "모델 배포는 일반적으로 Flask와 같은 웹 프레임워크를 사용하여 API를 구축하는 형태로 진행합니다. 예를 들어, 모델을 로드한 후 요청을 받아 예측 결과를 반환하는 RESTful API를 구성할 수 있습니다. 먼저, 필요한 라이브러리와 모델을 임포트하고, Flask 애플리케이션을 초기화 한 뒤, '/predict' 엔드포인트를 생성하여 POST 요청을 통해 받은 데이터로 모델을 호출해 결과를 리턴하는 방식으로 구현할 수 있습니다."
        },
        {
            "question": "Docker와 Kubernetes를 활용하여 머신러닝 서비스를 배포한 경험에 대해 설명해 주세요.",
            "answer": "Docker를 사용하여 머신러닝 모델의 환경을 컨테이너화하고, 이를 Kubernetes 클러스터에 배포하는 과정을 경험했습니다. 먼저 Dockerfile을 작성하여 필요한 라이브러리와 애플리케이션 코드를 포함한 이미지를 생성했습니다. 이후 Helm Chart를 사용하여 Kubernetes에 배포를 구성하고, 스케일링, 로드 밸런싱 등의 기능을 활용하여 운영하였습니다."
        },
        {
            "question": "Jenkins를 이용한 CI/CD 파이프라인 구축 경험을 설명해 주세요.",
            "answer": "Jenkins를 통해 코드를 GitLab에 푸시할 때마다 자동으로 빌드 및 테스트를 진행하도록 설정했습니다. Jenkinsfile을 작성하여 각 단계에서 필요한 스크립트를 명시하고, 성공할 경우 Kubernetes 클러스터에 자동으로 배포되는 파이프라인을 구성하였습니다. 이를 통해 배포 시간을 단축시키고

In [17]:
import json

data = json.loads(output)
print(type(data))
print(data)

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']

print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
{'hard_skill': [{'question': 'Python 프로그래밍 언어를 사용하여 머신러닝 모델을 배포하기 위한 코드를 어떻게 작성하겠습니까?', 'answer': "모델 배포는 일반적으로 Flask와 같은 웹 프레임워크를 사용하여 API를 구축하는 형태로 진행합니다. 예를 들어, 모델을 로드한 후 요청을 받아 예측 결과를 반환하는 RESTful API를 구성할 수 있습니다. 먼저, 필요한 라이브러리와 모델을 임포트하고, Flask 애플리케이션을 초기화 한 뒤, '/predict' 엔드포인트를 생성하여 POST 요청을 통해 받은 데이터로 모델을 호출해 결과를 리턴하는 방식으로 구현할 수 있습니다."}, {'question': 'Docker와 Kubernetes를 활용하여 머신러닝 서비스를 배포한 경험에 대해 설명해 주세요.', 'answer': 'Docker를 사용하여 머신러닝 모델의 환경을 컨테이너화하고, 이를 Kubernetes 클러스터에 배포하는 과정을 경험했습니다. 먼저 Dockerfile을 작성하여 필요한 라이브러리와 애플리케이션 코드를 포함한 이미지를 생성했습니다. 이후 Helm Chart를 사용하여 Kubernetes에 배포를 구성하고, 스케일링, 로드 밸런싱 등의 기능을 활용하여 운영하였습니다.'}, {'question': 'Jenkins를 이용한 CI/CD 파이프라인 구축 경험을 설명해 주세요.', 'answer': 'Jenkins를 통해 코드를 GitLab에 푸시할 때마다 자동으로 빌드 및 테스트를 진행하도록 설정했습니다. Jenkinsfile을 작성하여 각 단계에서 필요한 스크립트를 명시하고, 성공할 경우 Kubernetes 클러스터에 자동으로 배포되는 파이프라인을 구성하였습니다. 이를 통해 배포 시간을 단축시키고 코드 품질을 높일 수 있었습니다.'}], 'soft_skill': [{'question': '팀원과의 갈등이 생겼을 때 어떻게 대처하나요?', 'answer': '갈등이 발생했을 때는 먼저 상